# Projeto 2 - Ciência dos Dados

Nome: Henrique Thomé

Nome: _____

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [38]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [39]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [40]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])


___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [41]:
#Produto escolhido:
produto = 'Motorola One'

#Quantidade mínima de mensagens capturadas:
n = 600
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [42]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [43]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.drop_duplicates(subset ="Treinamento", keep = False, inplace = True) 
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.drop_duplicates(subset ="Teste", keep = False, inplace = True) 
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

In [44]:
dados_teste = pd.read_excel('Motorola One.xlsx',sheet_name='Teste')
dados_treinamento = pd.read_excel('Motorola One.xlsx',sheet_name='Treinamento')

In [45]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    punctuation = '[!\-.:?;,]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed    

In [55]:
df_rel = dados_treinamento[dados_treinamento['Class']==1]
textao = ' '.join(df_rel['Treinamento'])
textao_limpo = cleanup(textao)
lista_palavras = textao_limpo.split()
freq_abs_rel = pd.Series(lista_palavras).value_counts()
contagem_rel= dict(freq_abs_rel)

In [56]:
df_ir = dados_treinamento[dados_treinamento['Class']==0]
textaoir = ' '.join(df_ir['Treinamento'])
textao_limpoir = cleanup(textaoir)
lista_palavrasir = textao_limpoir.split()
freq_abs_ir = pd.Series(lista_palavrasir).value_counts()
contagem_ir= dict(freq_abs_ir)

In [57]:
def total_palavras(dicionario):
    contador = 0
    for e in dicionario:
        contador+=dicionario[e]
    return contador

### Contando o total de palavras para fazer o Naive-Bayes

In [58]:
total_palavras_ir = total_palavras(contagem_ir)
total_palavras_rel = total_palavras(contagem_rel)
total_geral = len(contagem_ir) + len(contagem_rel)

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

### Montando um dicionario com a probabilidade das palavras que já existem

In [59]:
prob_rel = {}
for e in contagem_rel:
    prob_rel[e] = (contagem_rel[e] + 1)/(total_palavras_rel + total_geral)
prob_ir = {}
for e in contagem_ir:
    prob_ir[e] = (contagem_ir[e]+1)/(total_palavras_ir + total_geral)

### Montando um dicionario com a prob das palavras no df teste

In [60]:
new_df = dados_teste['Teste']
text = ' '.join(new_df)
textao_lim = cleanup(text)
lista_pala = textao_lim.split()
freq = pd.Series(lista_pala).value_counts()
cont= dict(freq)

In [61]:
test_rel = {}
test_ir = {}
for e in cont.keys():
    if e not in prob_rel:
        test_rel[e] = 1/(total_palavras_rel + total_geral)
    else:
        test_rel[e] = (contagem_rel[e] + 1)/(total_palavras_rel + total_geral)
for e in cont.keys():
    if e not in prob_ir:
        test_ir[e] = 1/(total_palavras_ir + total_geral)
    else:
        test_ir[e] = (contagem_ir[e] + 1)/(total_palavras_ir + total_geral)

In [62]:
x = new_df.values.tolist()

In [77]:
lista = []
lista_nova = []
for e in x:
    lista.append(cleanup(e))
for e in lista:
    lista_nova.append(e.split())

['motorola', 'one', 'zoom', 'vem', 'com', '48mp', 'e', 'zoom', '3x', 'https', '//t', 'co/1eqg40fm1a']


In [87]:
resultado = []
contador = 0 
for e in lista_nova:
    contador+=1
    re = 1
    ir = 1
    for x in e:
        re*=test_rel[x]
        ir*= test_ir[x]
    if re > ir:
                resultado.append(1)
    else:
                resultado.append(0)



___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [93]:
resultado_certo = (dados_teste['Class'].tolist())

In [95]:
positivo = 0
negativo = 0
falso_positivo = 0
falso_negativo = 0
for i in range(0,len(resultado_certo)):
    if resultado_certo[i] == 0 and resultado[i] == 0:
        negativo +=1
    elif resultado_certo[i] == 1 and resultado[i] == 1:
        positivo +=1
    elif resultado_certo[i] == 1 and resultado[i] == 0:
        falso_negativo +=1
    elif resultado_certo[i] == 0 and resultado[i] == 1:
        falso_positivo+=1

In [108]:
len(dados_teste[dados_teste['Class']==0])

161

In [110]:
print('Positivo',positivo/203)
print('Negativo',negativo/203)
print('Falso Negativo',falso_negativo/203)
print('Falso Positivo',falso_positivo/203)
print('Acuracia',(positivo+negativo)/203)
print('total de negativos na base teste',len(dados_teste[dados_teste['Class']==0])/203)

Positivo 0.1477832512315271
Negativo 0.6059113300492611
Falso Negativo 0.059113300492610835
Falso Positivo 0.18719211822660098
Acuracia 0.7536945812807881
total de negativos na base teste 0.7931034482758621


___
### Concluindo

Nosso classificador chegou a uma acurrácia de aproximadamente 75%. Esse resultado, em príncipio parece bastante satisfatório, uma vez que não temos uma grande quantidade de dados e a limpeza do texto está relativamente rudimentar. Ao analizarmos mais a fundo os dados é possível observar que a grande maioria dos dados para o treinamento foram tweets negativos (não relevantes). Assim pode-se pensar que o classificador poderia estar classificando a maioria das frases como negativas e tendo, por isso, um bom desempenho. 
Esse hipótese mostra-se errada, uma vez que a maior parte dos erros foi de falsos positivos, indicando que a maquina não classificou do modo como pensávamos, mostrando assim que o computador foi capaz de reconhecer padrões nos comentários. Apesar disso, o programa mostra-se ineficiente, uma vez que se ele classificasse todos os tweets como negativos teria uma porcentagem maior de acertos.

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

##### Para aperfeiçoar o classificador é possível retirar links, @ e rts, pois esses não influenciam a classificação da palavras. Além disso para o classificador ser o mais preciso possível era necessário selecionar só as palavras chaves de cada tweet, tirando assim tudo que não importa. Para fazer isso, um novo classificador do tipo naive-bayes poderia ser criado, com um database classificando apenas as palavras chaves do texto. Dessa forma o primeiro programa selecionaria apenas as palavras chaves, enquanto o segundo as classificaria em irrelevante e relevante.
##### Ao analisar as mensagem foi possível perceber que o número de frases irrelevantes e muito maior que o de relevantes. Isso pode ocasionar resultados falsos, uma vez que se o classificador der sempre irrelevante ele tem grandes chances de estar certo.
##### Não podemos usar o classificador para gerar mais amostras de treinamento, pois se isso fosse teriamos uma amostra viciada, ou seja, o dicionario com as probabilidades de relevantes e irrelevantes nunca seriam alterados e, portanto os resultados seriam os mesmos
#### Existem milhares de aplicações para programas como esse. Um exemplo disso seria um analizador sentimental para avaliar contratações por times de futebol. Dessa forma a diretoria do clube saberia se o possível reforço tem uma boa aceitação pela torcida. Também existem milhares de aplicações para o governo, que por meio do programa poderia saber se a população está gostando do seu trabalho ou de uma determinada proposta de lei

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**